下載相關資料的小魔法

In [ ]:
!curl "https://gist.githubusercontent.com/penut85420/5b383ee875f66cfba70c46ad0e2dd21b/raw/fbd21d468add3ad99370e23ed9dcbf10aa7740c9/kana-spell.json" -o "kana-spell.json"

匯入相關套件

In [ ]:
# %pip install gradio -Uq
import json
import random

import gradio as gr

讀取 JSON 檔案

In [ ]:
def load_json(file_path):
    with open(file_path, "rt", encoding="UTF-8") as fp:
        return json.load(fp)


data = load_json("kana-spell.json")

範例用法

In [ ]:
selected = ["handakuon"]
# selected = ["seion", "dakuon", "handakuon", "youon"]  # 全選
category = [key for category in selected for key in data["category"][category]]
char_list = [ch for key in category for ch in data["hiragana"][key]]
char_list

分頁與切換分頁範例

In [ ]:
with gr.Blocks() as app:
    with gr.Tabs(selected=1) as tabs:
        with gr.Tab("分頁 A", id=0):
            gr.Markdown("這是分頁 A")
        with gr.Tab("分頁 B", id=1):
            gr.Markdown("這是分頁 B")

    btn_a = gr.Button("切換分頁 A")
    btn_b = gr.Button("切換分頁 B")

    btn_a.click(lambda: gr.Tabs(selected=0), outputs=tabs)
    btn_b.click(lambda: gr.Tabs(selected=1), outputs=tabs)

    app.launch()

準備資料

In [ ]:
data = load_json("kana-spell.json")

hiragana = data["hiragana"]
katakana = data["katakana"]
category = data["category"]
spell = data["spell"]

Gradio 元件宣告

In [ ]:
with gr.Blocks() as app:
    st_queue = gr.State(None)

    # 全部放在一個 gr.Tabs 裡面
    with gr.Tabs(selected=0) as tabs:
        with gr.Tab(label="設定", id=0):
            # 使用 gr.Group 可以把這區的元件都黏在一起沒有空隙
            with gr.Group():
                # 這一區都是可以選擇的假名種類
                chk_kana = gr.CheckboxGroup(["平假名", "片假名"], value=["平假名"], label="假名")
                chk_seion = gr.CheckboxGroup(category["seion"], value=["a"], label="清音")

                # 濁音跟半濁音比較少，放在同一排
                with gr.Row():
                    chk_dakuon = gr.CheckboxGroup(category["dakuon"], label="濁音")
                    chk_handakuon = gr.CheckboxGroup(category["handakuon"], label="半濁音")

                chk_youon = gr.CheckboxGroup(category["youon"], label="拗音")

            with gr.Row():
                # 背到後期時會需要一次選取很多項目
                btn_select_all = gr.Button("全選")
                btn_select_none = gr.Button("全不選")
            btn_start = gr.Button("開始測驗")

        with gr.Tab(label="測驗", id=1):
            with gr.Group():
                # 不可以讓使用者修改題目
                txt_test = gr.Textbox(label="題目", interactive=False)

                # 讓使用者作答的地方
                txt_input = gr.Textbox(label="作答", submit_btn=True)

    # 這是個用來除錯的輔助元件
    # 把 visible 設成 False 就不會顯示出來
    # 這樣就不用修改原本的程式碼了
    debug = gr.TextArea(label="Debug", visible=True)

    app.launch()

開始測驗時選擇題目

In [ ]:
def start_test(kana, seion, dakuon, handakuon, yoon):
    # seion 等參數會傳入像是 ["a", "ka"] 等列表
    # 使用 * 將這些列表展開成一維列表
    category = [*seion, *dakuon, *handakuon, *yoon]

    # 從平假名或片假名資料中取出對應的假名，全部放在 char_list 裡面
    char_list = list()
    char_list += [ch for k in category for ch in hiragana[k]] if "平假名" in kana else []
    char_list += [ch for k in category for ch in katakana[k]] if "片假名" in kana else []

    # 如果 char_list 是空的，則拋出錯誤
    if not char_list:
        raise gr.Error("請至少選擇一個類別")

    # 隨機打亂 char_list 的順序
    random.shuffle(char_list)

    # 取出第一個假名
    char = char_list.pop(0)

    # 回傳 char 給 txt_test 用來顯示題目
    # 第一個 char_list 回傳給 st_queue 用來紀錄題目狀態
    # 第二個 char_list 回傳給 debug 用來檢查
    # 回傳 gr.Tabs(selected=1) 來切換到測驗分頁
    return char, char_list, char_list, gr.Tabs(selected=1)

檢查作答是否正確

In [ ]:
def check_answer(txt_test, txt_input):
    # 將輸入的拼音轉為小寫並去除前後空白
    txt_input = str.lower(txt_input).strip()

    # 如果 txt_input 符合任何一種拼音，則正確
    if txt_input in spell[txt_test]:
        gr.Info("正確！")

    # 如果拼音不正確，提示使用者正確的答案有哪些可能
    else:
        answer = ", ".join(spell[txt_test])
        gr.Info(f"錯誤，正確答案為 {answer}")

    # 回傳 None 來清空 txt_input 的內容
    return None

Typing Hint 偷吃步

In [ ]:
def foo(a: list, b: dict, c: str):
    a.pop()
    b.items()
    c.lower()


def bar(a, b, c):
    list.pop(a)
    dict.items(b)
    str.lower(c)

顯示下一題

In [ ]:
def next_char(st_queue):
    # 若 st_queue 是空的，則顯示測驗結束的訊息
    if not st_queue:
        gr.Info("測驗結束！")
        return None, None, None

    # 繼續從 st_queue 中取出下一個假名
    char = list.pop(st_queue, 0)

    # 分別回傳給 txt_test, st_queue, debug 等元件
    return char, st_queue, st_queue

全選與全不選

In [ ]:
def select_all():
    return (
        ["平假名", "片假名"],
        category["seion"],
        category["dakuon"],
        category["handakuon"],
        category["youon"],
    )


def select_none():
    return [], [], [], [], []

In [ ]:
# font = gr.themes.GoogleFont("Kiwi Maru")
# font = gr.themes.GoogleFont("Rampart One")
font = gr.themes.GoogleFont("DotGothic16")
theme = gr.themes.Ocean(font=font)

with gr.Blocks(theme=theme) as app:
    st_queue = gr.State(None)

    # region define layout
    with gr.Tabs(selected=0) as tabs:
        with gr.Tab(label="設定", id=0):
            with gr.Group():
                chk_kana = gr.CheckboxGroup(["平假名", "片假名"], value=["平假名"], label="假名")
                chk_seion = gr.CheckboxGroup(category["seion"], value=["a"], label="清音")
                with gr.Row():
                    chk_dakuon = gr.CheckboxGroup(category["dakuon"], label="濁音")
                    chk_handakuon = gr.CheckboxGroup(category["handakuon"], label="半濁音")
                chk_youon = gr.CheckboxGroup(category["youon"], label="拗音")

            with gr.Row():
                btn_select_all = gr.Button("全選")
                btn_select_none = gr.Button("全不選")
            btn_start = gr.Button("開始測驗")

        with gr.Tab(label="測驗", id=1):
            with gr.Group():
                txt_test = gr.Textbox(label="題目", interactive=False)
                txt_input = gr.Textbox(label="作答", submit_btn=True)

    debug = gr.TextArea(label="Debug", visible=False)
    # endregion

    # region register events
    btn_start.click(
        start_test,
        [chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        [txt_test, st_queue, debug, tabs],
        show_progress="hidden",
    )

    txt_input.submit(
        check_answer,
        [txt_test, txt_input],
        txt_input,
        show_progress="hidden",
    ).then(
        next_char,
        st_queue,
        [txt_test, st_queue, debug],
        show_progress="hidden",
    )

    btn_select_all.click(
        select_all,
        outputs=[chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        show_progress="hidden",
    )

    btn_select_none.click(
        select_none,
        outputs=[chk_kana, chk_seion, chk_dakuon, chk_handakuon, chk_youon],
        show_progress="hidden",
    )
    # endregion

    app.launch()